In [1]:
import json
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
from statistics import mean, median, stdev, variance
import qgrid

In [2]:
# folder paths
results_folder = 'results/2020_05_11_ext3_2/'
gurobi_folder = 'Gurobi/'
ortools_folder = 'OrTools/'
gurobi_cold_folder = 'GurobiCold/'
extension1 = 'ext1/'
extension2 = 'ext2/'
extension3 = 'ext3/'
extension4 = 'ext4/'

# problem names
# p = set(f_name for f_name in gurobi_ext4_files)
p = set()

# solver 
solvers = {'or-tools':'OrTools/', 'gurobi-warm':'Gurobi/', 'gurobi-cold':'GurobiCold/'}

# extension
extensions = [1, 2, 3, 4]

for solver in solvers:
    solverFolder = solvers[solver]
    for extension in extensions:
        folder_path_extension = results_folder + solverFolder + f"ext{extension}/"
        if os.path.exists(folder_path_extension):
            files = set(f for f in os.listdir(folder_path_extension) if  f.endswith('.json'))
            #print(files)
            [p.add(f) for f in files]

# folder paths
f_paths = {}

for s in solvers:
    solver = solvers[s]
    f_paths[s] = {}
    for ext in extensions:
        f_paths[s][ext] = {}
        for problem in p:
            f_path = results_folder + solver + f"ext{ext}/" + problem
            if os.path.isfile(f_path):
                f_paths[s][ext][problem] = f_path

data = {'p':  [],
        'solver': [],
        'ext' : [],
        'clauses' : [],
        'steps': [],
        'total time [s]': [],
        '% hs [s]' : [],
        '% sat [s]' : [],
        '% grow [s]':[]
        }

for solver in f_paths:
    for ext in f_paths[solver]:
        for problem in f_paths[solver][ext]:
            f_path = f_paths[solver][ext][problem]

            # data parameters
            if '_random.json' in problem:
                data['ext'].append('2a')
                data['p'].append(problem.replace("_random",''))
            elif 'bestliteral.json' in problem:
                data['ext'].append('2b')
                data['p'].append(problem.replace("bestliteral",''))
            elif 'bestliteral_neg.json' in problem:
                data['ext'].append('2c')
                data['p'].append(problem.replace("bestliteral_neg",''))
            else:
                data['ext'].append(str(ext))
                data['p'].append(problem)
            
            data['solver'].append(solver)
            
            with open(f_path) as f:
                parsed_json = json.load(f)
            
            # data results
            data['clauses'].append(parsed_json['clauses'])
            data['steps'].append(parsed_json['steps'])
            tot_time = sum(parsed_json['t_hitting_set']) +sum(parsed_json['t_sat_check'])  + sum(parsed_json['t_grow'])
            data['total time [s]'].append(tot_time)
            data['% hs [s]'].append(round(100*sum(parsed_json['t_hitting_set'])/tot_time, 2))
            data['% sat [s]'].append(round(100*sum(parsed_json['t_sat_check'])/tot_time,2))
            data['% grow [s]'].append(round(100*sum(parsed_json['t_grow'])/tot_time, 2))
df = pd.DataFrame (data, columns = [column for column in data])
df = df.sort_values(["clauses", "steps", "total time [s]"], ascending = (False, True, True))
#print(df)
#df.to_excel("results_tiastower/2020_05_08.xlsx") 
#qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
#qgrid_widget

# Results
## RQ1: for or-tools as HS solver, extension 1 vs 2 ia ib ic vs 3

- For OR-tools as optimal hittingset solver, we can see that in all the cases, **extension3** performs better than **extension 2**. 
- **Extension 1** provides results in a considerably longer time even for small instances...

**Conclusion RQ1**

Best extension = `extension 2b`

In [5]:
# filter columns 
selected_columns = ['p','solver', 'ext', 'clauses', 'steps', 'total time [s]']
#print(df)
# data filter
df_ortools_ext123 = df[selected_columns]
df_ortools_ext123 = df_ortools_ext123[(df_ortools_ext123['ext'].isin(['2a', '2b', '2c', '3']))]
df_ortools_ext123 = df_ortools_ext123[(df_ortools_ext123.solver == 'or-tools')]
df_ortools_ext123 = df_ortools_ext123.sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))
# display table
df_ortools_ext123
#qgrid_widget = qgrid.show_grid(df_ortools_ext123, show_toolbar=True)
#qgrid_widget

,p,solver,ext,clauses,steps,total time [s]
12,par8-1-c.json,or-tools,2c,250,2,0.007527
6,par8-1-c.json,or-tools,2b,250,2,0.008216
15,par8-1-c.json,or-tools,2a,250,2,0.008245
14,dubois22.json,or-tools,2b,176,179,10.454270
13,dubois22.json,or-tools,2a,176,193,11.509692
2,dubois22.json,or-tools,2c,176,193,11.760376
8,dubois21.json,or-tools,2b,168,180,9.752085
10,dubois21.json,or-tools,2a,168,195,11.194850
5,dubois21.json,or-tools,2c,168,195,11.269691
1,aim-100-1_6-no-1.json,or-tools,2b,160,132,3.513045


## RQ2: for the best of RQ1, compare or-tools vs gurobi

We compare the results of `extension 2b` for the 3 solvers : 

- `Or-Tools`: Or-tools optimal hitting set solver
- `Gurobi Warm Start`: Model is built on all variables and for every new Minimum Correction Set found, we had a new constraint corresponding to a new set in the collection of sets to hit.
- `Gurobi Cold Start`: The full model is rebuilt for every optimal hitting set call

**Results are sorted by decreasing number of clauses, Increasing problem name, total solving time**

- We can see that for `extension 2b` the total solving time `total time [s]` is always the smallest with gurobi using a warm start. 
- For cold starts, or-tools performs better than gurobi.

**Conclusion RQ1**

Best combo = `gurobi warm start + extension 2b`

In [6]:
# filter columns 
selected_columns = ['p','solver', 'ext', 'clauses', 'steps', 'total time [s]']

# data filter
df_solver_ext3 = df[selected_columns][(df['ext'].isin(['2b','3']))]
df_solver_ext3 = df_solver_ext3[(df_solver_ext3['solver'].isin(['or-tools', 'gurobi-warm', 'gurobi-cold']))]
df_solver_ext3 = df_solver_ext3.sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))
df_solver_ext3
# display marked-up table
#qgrid_widget = qgrid.show_grid(df_solver_ext3, show_toolbar=True)
#qgrid_widget

,p,solver,ext,clauses,steps,total time [s]
36,zebra_v155_c1135.json,gurobi-warm,3,1160,689,554.199746
24,par8-1-c.json,gurobi-warm,2b,250,2,0.002541
6,par8-1-c.json,or-tools,2b,250,2,0.008216
38,par8-1-c.json,gurobi-warm,3,250,2,0.070206
57,par8-1-c.json,gurobi-cold,2b,250,2,0.333014
32,dubois22.json,gurobi-warm,2b,176,179,0.326379
41,dubois22.json,gurobi-warm,3,176,421,1.552734
14,dubois22.json,or-tools,2b,176,179,10.454270
65,dubois22.json,gurobi-cold,2b,176,179,14.659949
26,dubois21.json,gurobi-warm,2b,168,179,0.295172


## RQ3: for the best of RQ2, compare  with maxsat

We compare the results :
- `Gurobi warm start + extension2b` vs 
- `Gurobi warm start + Max Sat` 

We see that for the medium instances, extension 3 takes too much time (> 1 day).

1. bf0432-007
2. zebra_v155_c1135

For smaller instances, we see that the MaxSat solution takes :
- less steps 
- sometimes slower on smaller instances than the greedy approach of extension 2b.

**Conclusion**

Overall the maxsat solution implemented by https://pysathq.github.io/docs/html/api/examples/rc2.html:

    Alexey Ignatiev, António Morgado, Joao Marques-Silva. RC2: An Efficient MaxSAT Solver. MaxSAT Evaluation 2018. JSAT 11. 2019. pp. 53-64

shows very good results and performs better on medium instances.

**Best combo:**
- Solver: **Gurobi with warm start**
- Extension : **Maxsat**

In [8]:
# filter columns 
selected_columns = ['p','solver', 'ext', 'clauses', 'steps', 'total time [s]']

# data filter
df_warm_start = df[selected_columns]
#print(df_warm_start)
df_warm_start = df_warm_start[df_warm_start.solver == 'gurobi-warm']
df_warm_start = df_warm_start[df_warm_start['ext'].isin(['3', '2b', '4'])]
df_warm_start = df_warm_start.sort_values(
    ["clauses", "p", "total time [s]"], 
    ascending = (False, True, True))

df_warm_start

,p,solver,ext,clauses,steps,total time [s]
46,bf0432-007.json,gurobi-warm,4,3667,1274,112.622693
43,zebra_v155_c1135.json,gurobi-warm,4,1160,544,136.784257
36,zebra_v155_c1135.json,gurobi-warm,3,1160,689,554.199746
24,par8-1-c.json,gurobi-warm,2b,250,2,0.002541
45,par8-1-c.json,gurobi-warm,4,250,2,0.016326
38,par8-1-c.json,gurobi-warm,3,250,2,0.070206
32,dubois22.json,gurobi-warm,2b,176,179,0.326379
49,dubois22.json,gurobi-warm,4,176,176,0.443707
41,dubois22.json,gurobi-warm,3,176,421,1.552734
26,dubois21.json,gurobi-warm,2b,168,179,0.295172
